In [1]:
import pandas as pd
import numpy as np
import re
from re import search
import codecs
import sys
import string

In [2]:
stop_words = []
prefixes = []
postfixes = []
verb_roots = []
common_words = []
plural_singular = []


def read_dataset(path, name):
    df = pd.read_excel(path + name)
    return df

def read_files():
    path = 'files/'

    #stop_words
    name = 'stop_words.txt'
    f = open(path + name, 'r', encoding='utf-8')
    Lines = f.read().splitlines()
    # Strips the newline character
    for line in Lines:
        stop_words.append(line)
    f.close()

    #prefixes
    name = 'prefixes.txt'
    f = open(path + name, 'r', encoding='utf-8')
    Lines = f.read().splitlines()
    # Strips the newline character
    for line in Lines:
        prefixes.append(line)
    f.close()

    #postfixes
    name = 'postfixes.txt'
    f = open(path + name, 'r', encoding='utf-8')
    Lines = f.read().splitlines()
    # Strips the newline character
    for line in Lines:
        postfixes.append(line)
    f.close()

    # verb_roots
    name = 'verb.txt'
    f = open(path + name, 'r', encoding='utf-8')
    Lines = f.read().splitlines()
    count = 0
    # Strips the newline character
    for line in Lines:
        line = line.split('-')
        verb_roots.append(line)
    f.close()

    #common_words
    name = 'common_words.txt'
    f = open(path + name, 'r', encoding='utf-8')
    Lines = f.read().splitlines()
    # Strips the newline character
    for line in Lines:
        common_words.append(line)
    f.close()

    #plural_singular
    name = 'plural_singular.txt'
    f = open(path + name, 'r', encoding='utf-8')
    Lines = f.read().splitlines()
    count = 0
    # Strips the newline character
    for line in Lines:
        line = line.split('-')
        plural_singular.append(line)
    f.close()

def delete_punctuations(doc):
    punctuations = '،:؛؟!»«()[]"*,{.}@!?/'
    edited_doc = doc.translate(str.maketrans('', '', punctuations))
    edited_doc = doc.translate(str.maketrans('', '', string.punctuation))
    
    return edited_doc

def delete_stopWords(doc):
    edited_doc = doc
    for s in stop_words:
        my_regex = r"\b"+s+r"\b"
        edited_doc = re.sub(my_regex , "", edited_doc)
    return edited_doc

def delete_highFrequencyWords(inverted_indexes):
    df_temp = df.copy()
    for i in inverted_indexes:
        if len(i[1])/len(df_temp) > 0.8:
            inverted_indexes.remove(i)
    return inverted_indexes

def delete_postfixes(doc):
    edited_doc = doc
    for p in postfixes:
        my_regex = p + r"\b"
        edited_doc = re.sub(my_regex , "", edited_doc)
    return edited_doc

def delete_prefixes(doc):
    edited_doc = doc
    for p in postfixes:
        my_regex = r"\b" + p
        edited_doc = re.sub(my_regex , "", edited_doc)
    return edited_doc

def replaceWithRoot(tokens):   
    for i in range(0, len(tokens)):
        for root in verb_roots:
            if search(root[0], tokens[i]) or search(root[1], tokens[i]):
                common = False
                for c in common_words:
                    if c == tokens[i]:
                        common = True
                        print("common")
                        break
                if common == False:
                    print("root")
                    tokens[i] = tokens[i].replace(tokens[i], root[2])

    return tokens

def replaceArabicWords(doc):
    doc = doc.replace('ك', 'ک')
    doc = doc.replace('ئ', 'ی')
    doc = doc.replace('ي', 'ی')
    doc = doc.replace('ؤ', 'و')
    doc = doc.replace('هٔ', 'ه')
    doc = doc.replace('ة', 'ه')
    doc = doc.replace('آ', 'ا')
    doc = doc.replace('أ', 'ا')
    doc = doc.replace('إ', 'ا')
    return doc

def pluralToSingular(tokens):
    for i in range(0, len(tokens)):
        for ps in plural_singular:
            if search(ps[0], tokens[i]):
                tokens[i] = tokens[i].replace(tokens[i], ps[1])
    return tokens

def tokenize(df):
    content = df.content

    tokens = []
    for i in range(0, content.size):
        doc = content[i]
        # 68000 tokens
        doc = delete_punctuations(doc)
        # 50000 tokens
        doc = delete_stopWords(doc)

        doc = replaceArabicWords(doc)

        doc = delete_postfixes(doc)

        doc = delete_prefixes(doc)
        
        tokenized_doc = doc.split()
        tokenized_doc = replaceWithRoot(tokenized_doc)
        tokenized_doc = pluralToSingular(tokenized_doc)

        for token in tokenized_doc:
            temp = []
            temp.append(token)
            temp.append(df.id[i])
            tokens.append(temp)
    tokens.sort()
    return tokens

def create_inverted_indexes(tokens):
    inverted_indexes = []
    doc_temp = []
    token_temp = ""
    for token in tokens:
        if token[0] == token_temp:
            doc_temp.append(token[1])
        else:
            temp = []
            temp.append(token_temp)
            doc_temp = set(doc_temp)
            temp.append(doc_temp)
            inverted_indexes.append(temp)
            doc_temp = []

        token_temp = token[0]
        # if len(doc_temp) > 0 and token[1] != doc_temp[-1]:
        doc_temp.append(token[1])
    
    inverted_indexes = delete_highFrequencyWords(inverted_indexes)
    return inverted_indexes

def search_token(inverted_indexes, token_name):
    for token in inverted_indexes:
        if token[0] == token_name:
            return token[1]
    print("this token not exist in our database")

def print_res(df, res):
    p_res = pd.DataFrame()

    p_res = pd.merge(df, res, on=['id'], how='inner')
    our_res = p_res[['id', 'rank', 'url']]
    our_res = our_res.sort_values(["rank"], ascending=False)

    print(our_res)
    

def query_processing(df, inverted_indexes, query):
    res = []
    docs_id = pd.DataFrame()


    query = delete_punctuations(query)
    query = delete_stopWords(query)
    query = delete_postfixes(query)
    query = delete_prefixes(query)
    query = replaceArabicWords(query)

    tokenized_query = query.split()
    tokenized_query = replaceWithRoot(tokenized_query)
    tokenized_query = pluralToSingular(tokenized_query)

    if len(tokenized_query) == 0:
        print("It looks like there aren't many great matches for your search")
        return


    for i in range (0 ,len(tokenized_query)):
        temp_docs_id = pd.DataFrame(search_token(inverted_indexes, tokenized_query[i]))
        docs_id = pd.DataFrame(docs_id.append(temp_docs_id))

    docs_id = docs_id[0].value_counts().reset_index()
    docs_id.columns = ['id', 'rank']
    print_res(df, docs_id)
    


In [3]:
df = read_dataset("datasets/", "IR_Spring2021_ph12_7k.xlsx")
read_files()

In [4]:
tokens = tokenize(df)
inverted_indexes = create_inverted_indexes(tokens)


common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common
common

In [5]:
print(len(tokens))
print(len(inverted_indexes))

# for i in range(5000, 5100):
#     print(inverted_indexes[i][0])
#     print(inverted_indexes[i][1])

1701668
49324


In [32]:
search_token(inverted_indexes, "مید")

{27,
 66,
 94,
 184,
 209,
 210,
 211,
 234,
 243,
 250,
 265,
 307,
 403,
 412,
 414,
 441,
 481,
 487,
 493,
 499,
 546,
 548,
 555,
 558,
 578,
 586,
 602,
 612,
 614,
 632,
 640,
 641,
 649,
 654,
 658,
 663,
 676,
 694,
 698,
 704,
 708,
 711,
 747,
 754,
 760,
 800,
 806,
 817,
 877,
 893,
 895,
 900,
 902,
 912,
 914,
 921,
 923,
 928,
 946,
 951,
 962,
 983,
 1006,
 1007,
 1019,
 1022,
 1026,
 1028,
 1037,
 1046,
 1063,
 1068,
 1080,
 1086,
 1094,
 1104,
 1124,
 1130,
 1138,
 1174,
 1211,
 1213,
 1223,
 1240,
 1288,
 1311,
 1345,
 1353,
 1354,
 1355,
 1357,
 1358,
 1363,
 1368,
 1372,
 1379,
 1397,
 1400,
 1401,
 1418,
 1456,
 1467,
 1503,
 1506,
 1536,
 1540,
 1552,
 1578,
 1603,
 1610,
 1630,
 1637,
 1644,
 1662,
 1663,
 1672,
 1717,
 1778,
 1882,
 1892,
 1948,
 1962,
 2048,
 2099,
 2137,
 2149,
 2252,
 2361,
 2458,
 2496,
 2532,
 2651,
 2696,
 2704,
 2705,
 2714,
 2764,
 2769,
 2968,
 3031,
 3128,
 3152,
 3225,
 3257,
 3529,
 3553,
 3661,
 3665,
 3758,
 3767,
 3937,
 3947,
 

In [31]:
query = "مید"
query_processing(df, inverted_indexes, query)


       id  rank                                                url
0      27     1  https://www.isna.ir/news/99011005164/روایت-روز...
128  2458     1  https://www.isna.ir/news/99112619458/بررسی-طرح...
119  1892     1  https://www.isna.ir/news/99041511345/ساماندهی-...
120  1948     1  https://www.isna.ir/news/99051309792/دستور-داد...
121  1962     1  https://www.isna.ir/news/99051813039/شمخانی-نم...
..    ...   ...                                                ...
64   1019     1  https://www.isna.ir/news/99122116413/قرعه-کشی-...
65   1022     1  https://www.isna.ir/news/99122216793/حذف-عسگری...
66   1026     1  https://www.isna.ir/news/99122418504/پایان-کار...
67   1028     1  https://www.isna.ir/news/99122620408/فولاد-سیر...
186  6890     1  https://www.isna.ir/news/98071713455/برای-اهدا...

[187 rows x 3 columns]
